In [ ]:
# LINK_DATASET = "/media/namvq/Data/chest_xray"
LINK_DATASET = "/kaggle/input/chest-xray-pneumonia/chest_xray"


In [14]:
"""Partition the data and create the dataloaders."""

from typing import List, Optional, Tuple

import torch
from omegaconf import DictConfig

from torch.utils.data import DataLoader, random_split, Subset
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Normalize, ToTensor
import os
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, Grayscale, ToTensor
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')  # Chuyển sang backend không cần GUI

print('BACKEND: ', matplotlib.get_backend())
NUM_WORKERS = 6
# def get_custom_dataset(data_path: str = "/media/namvq/Data/chest_xray"):
#     """Load custom dataset and apply transformations."""
#     transform = Compose([
#         Resize((100, 100)),
#         Grayscale(num_output_channels=1),
#         ToTensor()
#     ])
#     trainset = ImageFolder(os.path.join(data_path, 'train'), transform=transform)
#     testset = ImageFolder(os.path.join(data_path, 'test'), transform=transform)
#     return trainset, testset

# def get_custom_dataset(data_path: str = "/kaggle/input/chest-xray-pneumonia/chest_xray"):
#     """Load custom dataset and apply transformations."""
#     transform = transforms.Compose([
#         transforms.Resize((224, 224)),  # Kích thước ảnh cho EfficientNet
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],  # Mean chuẩn của ImageNet
#                              [0.229, 0.224, 0.225])  # Std chuẩn của ImageNet
#     ])
#     trainset = ImageFolder(os.path.join(data_path, 'train'), transform=transform)
#     testset = ImageFolder(os.path.join(data_path, 'test'), transform=transform)
#     return trainset, testset

# def get_custom_dataset(data_path: str = "/media/namvq/Data/chest_xray"):
#     """Load custom dataset and apply transformations."""
#     train_transform = transforms.Compose([
#         transforms.Resize((224, 224)),  # Kích thước ảnh cho EfficientNet
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],  # Mean chuẩn của ImageNet
#                              [0.229, 0.224, 0.225])  # Std chuẩn của ImageNet
#     ])
#     test_transform = transforms.Compose([
#         transforms.Resize((224, 224)),  # Kích thước ảnh cho EfficientNet
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],  # Mean chuẩn của ImageNet
#                              [0.229, 0.224, 0.225])  # Std chuẩn của ImageNet
#     ])
#     trainset = ImageFolder(os.path.join(data_path, 'train'), transform=train_transform)
#     testset = ImageFolder(os.path.join(data_path, 'test'), transform=test_transform)
#     return trainset, testset
# def get_custom_dataset(data_path: str = "/home/namvq1/Documents/chest_xray"):
#     """Load custom dataset and apply transformations."""
#     train_transform = transforms.Compose([
#         transforms.Resize((150, 150)),  # Kích thước ảnh cho VGG
#         transforms.RandomAffine(degrees=0, shear=10),
#         transforms.RandomHorizontalFlip(),
#         transforms.RandomResizedCrop(150, scale=(0.8, 1.0)),
#         transforms.RandomAffine(degrees=0, translate=(0.2, 0)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],  # Mean chuẩn của ImageNet
#                              [0.229, 0.224, 0.225])  # Std chuẩn của ImageNet
#     ])
#     test_transform = transforms.Compose([
#         transforms.Resize((150, 150)),  # Kích thước ảnh cho VGG
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],  # Mean chuẩn của ImageNet
#                              [0.229, 0.224, 0.225])  # Std chuẩn của ImageNet
#     ])
#     trainset = ImageFolder(os.path.join(data_path, 'train'), transform=train_transform)
#     testset = ImageFolder(os.path.join(data_path, 'test'), transform=test_transform)
#     return trainset, testset
#/home/namvq1/Documents/chest_xray

def get_custom_dataset(data_path: str = LINK_DATASET):
    #For resnet 
    """Load custom dataset and apply transformations."""
    train_transform = transforms.Compose([
        transforms.Resize(256),  # Kích thước ảnh cho Resnet
        transforms.RandomAffine(degrees=0, shear=10),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomAffine(degrees=0, translate=(0.2, 0)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],  # Mean chuẩn của ImageNet
                             [0.229, 0.224, 0.225])  # Std chuẩn của ImageNet
    ])
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Kích thước ảnh cho VGG
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],  # Mean chuẩn của ImageNet
                             [0.229, 0.224, 0.225])  # Std chuẩn của ImageNet
    ])
    trainset = ImageFolder(os.path.join(data_path, 'train'), transform=train_transform)
    testset = ImageFolder(os.path.join(data_path, 'test'), transform=test_transform)
    return trainset, testset



def prepare_dataset_for_centralized_train(batch_size: int, val_ratio: float = 0.1, seed: int = 42):
    trainset, testset = get_custom_dataset()
    # Split trainset into trainset and valset
    num_train = int((1 - val_ratio) * len(trainset))
    num_val = len(trainset) - num_train
    trainset, valset = random_split(trainset, [num_train, num_val], torch.Generator().manual_seed(seed))

    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
    valloader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS)

    print(f'Number of train samples: {len(trainset)}, val samples: {len(valset)}, test samples: {len(testloader.dataset)}')
    return trainloader, valloader, testloader


def prepare_dataset(num_partitions: int, batch_size: int, val_ratio: float = 0.1, alpha: float = 100, seed: int = 42):
    """Load custom dataset and generate non-IID partitions using Dirichlet distribution."""
    trainset, testset = get_custom_dataset()
    
    # Split trainset into trainset and valset
    num_train = int((1 - val_ratio) * len(trainset))
    num_val = len(trainset) - num_train
    trainset, valset = random_split(trainset, [num_train, num_val], torch.Generator().manual_seed(seed))
    
    # Get labels for the entire trainset
    train_labels = np.array([trainset.dataset.targets[i] for i in trainset.indices])
    
    # Generate Dirichlet distribution for each class
    class_indices = [np.where(train_labels == i)[0] for i in range(len(np.unique(train_labels)))]
    partition_indices = [[] for _ in range(num_partitions)]
    
    for class_idx in class_indices:
        np.random.shuffle(class_idx)
        proportions = np.random.dirichlet(np.repeat(alpha, num_partitions))
        proportions = (np.cumsum(proportions) * len(class_idx)).astype(int)[:-1]
        class_partitions = np.split(class_idx, proportions)
        for i in range(num_partitions):
            partition_indices[i].extend(class_partitions[i])
    
    # Create Subsets for each partition
    trainsets = [Subset(trainset.dataset, indices) for indices in partition_indices]
    
    # Split valset into partitions
    partition_len_val = [len(valset) // num_partitions] * num_partitions
    for i in range(len(valset) % num_partitions):
        partition_len_val[i] += 1

    valsets = random_split(valset, partition_len_val, torch.Generator().manual_seed(seed))
    
    # Create DataLoaders for each partition
    trainloaders = [DataLoader(ts, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS) for ts in trainsets]
    valloaders = [DataLoader(vs, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS) for vs in valsets]
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS)

    # Calculate class distribution for each partition in trainloaders
    class_distributions = []
    for i, trainloader in enumerate(trainloaders):
        class_counts = Counter()
        for _, labels in trainloader:
            class_counts.update(labels.numpy())
        class_distributions.append(class_counts)
        print(f'Partition {i} class distribution: {dict(class_counts)}')

    # Plot class distribution
    partitions = range(num_partitions)
    class_0_counts = [class_distributions[i][0] for i in partitions]
    class_1_counts = [class_distributions[i][1] for i in partitions]

    bar_width = 0.5
    plt.figure(figsize=(12, 8))
    plt.bar(partitions, class_0_counts, bar_width, label='Class 0', color='blue')
    plt.bar(partitions, class_1_counts, bar_width, bottom=class_0_counts, label='Class 1', color='red')
    plt.xlabel('Partition')
    plt.ylabel('Number of Samples')
    plt.title('Class Distribution in Each Partition')
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f'Number of train samples: {len(trainset)}, val samples: {len(valset)}, test samples: {len(testloader.dataset)}')
    return trainloaders, valloaders, testloader

def prepare_partitioned_dataset(num_partitions: int, batch_size: int, val_ratio: float = 0.1, num_labels_each_party: int = 1, seed: int = 42):
    """Load custom dataset and generate partitions where each party has a fixed number of labels."""
    trainset, testset = get_custom_dataset()  # Load datasets

    # Split the trainset into trainset and valset based on the validation ratio
    num_train = int((1 - val_ratio) * len(trainset))
    num_val = len(trainset) - num_train
    trainset, valset = random_split(trainset, [num_train, num_val], generator=torch.Generator().manual_seed(seed))

    # Get labels for the entire trainset
    train_labels = np.array([trainset.dataset.targets[i] for i in trainset.indices])

    # Define partitions: each party has k labels
    num_labels = len(np.unique(train_labels))  # Assuming labels are 0 and 1 for binary classification
    times = [0 for i in range(num_labels)]
    contain = []
    #Phan label cho cac client
    for i in range(num_partitions):
        current = [i%num_labels]
        times[i%num_labels] += 1
        if num_labels_each_party > 1:
            current.append(1-i%num_labels)
            times[1-i%num_labels] += 1
        contain.append(current)
    print(times)
    print(contain)
    # Create Subsets for each partition

    partition_indices = [[] for _ in range(num_partitions)]
    for i in range(num_labels):
        idx_i = np.where(train_labels == i)[0]  # Get indices of label i in train_labels
        idx_i = [trainset.indices[j] for j in idx_i]  # Convert indices to indices in trainset
        # #print label of idx_i
        # print("Label of idx: ", i)
        # for j in range(len(idx_i)):
        #     idx_in_dataset = trainset.indices[idx_i[j]]
        #     print(trainset.dataset.targets[idx_in_dataset])
        np.random.shuffle(idx_i)
        split = np.array_split(idx_i, times[i])
        ids = 0
        for j in range(num_partitions):
            if i in contain[j]:
                partition_indices[j].extend(split[ids])
                ids += 1
    
    trainsets = [Subset(trainset.dataset, indices) for indices in partition_indices]

    # #print label of client 0
    # print("Client 0")
    # for i in range(len(trainsets[0])):
    #     print(trainsets[0][i][1])

    # Split valset into partitions
    partition_len_val = [len(valset) // num_partitions] * num_partitions
    for i in range(len(valset) % num_partitions):
        partition_len_val[i] += 1
    
    valsets = random_split(valset, partition_len_val, generator=torch.Generator().manual_seed(seed))

    # Create DataLoaders for each partition
    trainloaders = [DataLoader(ts, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS) for ts in trainsets]
    valloaders = [DataLoader(vs, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS) for vs in valsets]
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS)

    # Calculate class distribution for each partition in trainloaders
    class_distributions = []
    for i, trainloader in enumerate(trainloaders):
        class_counts = Counter()
        for _, labels in trainloader:
            class_counts.update(labels.numpy())
        class_distributions.append(class_counts)
        print(f'Partition {i} class distribution: {dict(class_counts)}')
    # Plot class distribution
    partitions = range(num_partitions)
    class_0_counts = [class_distributions[i][0] for i in partitions]
    class_1_counts = [class_distributions[i][1] for i in partitions]

    bar_width = 0.5
    plt.figure(figsize=(12, 8))
    plt.bar(partitions, class_0_counts, bar_width, label='Class 0', color='blue')
    plt.bar(partitions, class_1_counts, bar_width, bottom=class_0_counts, label='Class 1', color='red')
    plt.xlabel('Partition')
    plt.ylabel('Number of Samples')
    plt.title('Class Distribution in Each Partition')
    plt.legend()
    plt.grid(True)
    # plt.show()

    #  Lưu đồ thị vào thư mục running_outputs với tên data_partition
    output_dir = 'running_outputs'
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(os.path.join(output_dir, 'data_partition.png'))
    plt.close()



    print(f'Number of train samples: {len(trainset)}, val samples: {len(valset)}, test samples: {len(testloader.dataset)}')
    return trainloaders, valloaders, testloader

def prepare_imbalance_label_dirichlet(num_partitions: int, batch_size: int, val_ratio: float = 0.1, beta: float = 0.5, seed: int = 42):
    """Load custom dataset and generate partitions where each party has a fixed number of labels."""
    trainset, testset = get_custom_dataset()  # Load datasets

    # Split the trainset into trainset and valset based on the validation ratio
    num_train = int((1 - val_ratio) * len(trainset))
    num_val = len(trainset) - num_train
    trainset, valset = random_split(trainset, [num_train, num_val], generator=torch.Generator().manual_seed(seed))

    # Get labels for the entire trainset
    train_labels = np.array([trainset.dataset.targets[i] for i in trainset.indices])

    # Define partitions: each party has k labels
    num_labels = len(np.unique(train_labels))  # Assuming labels are 0 and 1 for binary classification
    min_size = 0
    min_require_size = 2

    N = len(trainset)


    while(min_size < min_require_size):
        partition_indices = [[] for _ in range(num_partitions)]
        for label in range(num_labels):
            idx_label = np.where(train_labels == label)[0]
            idx_label = [trainset.indices[j] for j in idx_label]
            np.random.shuffle(idx_label)

            proportions = np.random.dirichlet(np.repeat(beta, num_partitions))
            # proportions = np.array( [p * len(idx_j) < N/num_partitions] for p, idx_j in zip(proportions, partition_indices))
            proportions = np.array([p if p * len(idx_j) < N / num_partitions else 0 for p, idx_j in zip(proportions, partition_indices)])

            proportions = proportions / np.sum(proportions)
            proportions = (np.cumsum(proportions) * len(idx_label)).astype(int)[:-1]

            partition_indices = [idx_j + idx.tolist() for idx_j, idx in zip(partition_indices, np.split(idx_label, proportions))]
            min_size = min([len(idx_j) for idx_j in partition_indices])
        
    trainsets = [Subset(trainset.dataset, indices) for indices in partition_indices]

    partition_len_val = [len(valset) // num_partitions] * num_partitions
    for i in range(len(valset) % num_partitions):
        partition_len_val[i] += 1
    
    valsets = random_split(valset, partition_len_val, generator=torch.Generator().manual_seed(seed))

    trainloaders = [DataLoader(ts, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS) for ts in trainsets]
    valloaders = [DataLoader(vs, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS) for vs in valsets]
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS)

    class_distributions = []
    for i, trainloader in enumerate(trainloaders):
        class_counts = Counter()
        for _, labels in trainloader:
            class_counts.update(labels.numpy())
        class_distributions.append(class_counts)
        print(f'Partition {i} class distribution: {dict(class_counts)}')
    # Plot class distribution
    partitions = range(num_partitions)
    class_0_counts = [class_distributions[i][0] for i in partitions]
    class_1_counts = [class_distributions[i][1] for i in partitions]

    bar_width = 0.5
    plt.figure(figsize=(12, 8))
    plt.bar(partitions, class_0_counts, bar_width, label='Class 0', color='blue')
    plt.bar(partitions, class_1_counts, bar_width, bottom=class_0_counts, label='Class 1', color='red')
    plt.xlabel('Partition')
    plt.ylabel('Number of Samples')
    plt.title('Class Distribution in Each Partition')
    plt.legend()
    plt.grid(True)
    #  Lưu đồ thị vào thư mục running_outputs với tên data_partition
    output_dir = 'running_outputs'
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(os.path.join(output_dir, 'data_partition.png'))
    plt.close()

    print(f'Number of train samples: {len(trainset)}, val samples: {len(valset)}, test samples: {len(testloader.dataset)}')

    return trainloaders, valloaders, testloader



def apply_gaussian_noise(tensor, std_dev):
    noise = torch.randn_like(tensor) * std_dev
    return tensor + noise

# Hàm đảo ngược chuẩn hóa
def unnormalize_image(image_tensor, mean, std):
    # Đảo ngược Normalize: (image * std) + mean
    for t, m, s in zip(image_tensor, mean, std):
        t.mul_(s).add_(m)  # Thực hiện từng kênh
    return image_tensor

# Hàm hiển thị ảnh từ một tensor
def display_image(image_tensor, mean, std):
    # Đảo ngược chuẩn hóa
    image_tensor = unnormalize_image(image_tensor, mean, std)
    # Chuyển tensor thành NumPy array và điều chỉnh thứ tự kênh màu (CHW -> HWC)
    image_numpy = image_tensor.permute(1, 2, 0).numpy()
    # Cắt giá trị ảnh về phạm vi [0, 1] để hiển thị đúng
    image_numpy = image_numpy.clip(0, 1)
    # Trả về ảnh NumPy
    return image_numpy

def prepare_noise_based_imbalance(num_partitions: int, batch_size: int, val_ratio: float = 0.1, sigma: float = 0.05, seed: int = 42):
    """
    Chia du lieu ngau nhien va deu cho cac ben, sau do them noise vao cac ben
    moi ben i co noise khac nhau Gauss(0, sigma*i/N)
    """
    trainset, testset = get_custom_dataset()
    num_train = int((1 - val_ratio) * len(trainset))
    num_val = len(trainset) - num_train
    trainset, valset = random_split(trainset, [num_train, num_val], generator=torch.Generator().manual_seed(seed))

    indices = trainset.indices

    np.random.shuffle(indices)

    partition_indices = np.array_split(indices, num_partitions)

    train_partitions = []

    for i, part_indices in enumerate(partition_indices):
        partition_std_dev = sigma * (i + 1) / num_partitions
        partition_set = Subset(trainset.dataset, part_indices)
        
        noisy_samples = [apply_gaussian_noise(sample[0], partition_std_dev) for sample in partition_set]
        noisy_dataset = [(noisy_samples[j], trainset.dataset[part_indices[j]][1]) for j in range(len(part_indices))]
        # train_partitions.append((noisy_samples, [sample[1] for sample in partition_set]))
        train_partitions.append(noisy_dataset)
    trainloaders = [DataLoader(train_partitions[i], batch_size=batch_size, shuffle=True, num_workers=4) for i in range(num_partitions)]
    partition_len_val = [len(valset) // num_partitions] * num_partitions
    for i in range(len(valset) % num_partitions):
        partition_len_val[i] += 1
    
    valsets = random_split(valset, partition_len_val, generator=torch.Generator().manual_seed(seed))
    valloaders = [DataLoader(vs, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS) for vs in valsets]
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS)

####
    class_distributions = []
    for i, trainloader in enumerate(trainloaders):
        class_counts = Counter()
        for _, labels in trainloader:
            class_counts.update(labels.numpy())
        class_distributions.append(class_counts)
        print(f'Partition {i} class distribution: {dict(class_counts)}')
    
    partitions = range(num_partitions)
    class_0_counts = [class_distributions[i][0] for i in partitions]
    class_1_counts = [class_distributions[i][1] for i in partitions]

    bar_width = 0.5
    plt.figure(figsize=(12, 8))
    plt.bar(partitions, class_0_counts, bar_width, label='Class 0', color='blue')
    plt.bar(partitions, class_1_counts, bar_width, bottom=class_0_counts, label='Class 1', color='red')
    plt.xlabel('Partition')
    plt.ylabel('Number of Samples')
    plt.title('Class Distribution in Each Partition')
    plt.legend()
    plt.grid(True)
    # plt.show()
    #  Lưu đồ thị vào thư mục running_outputs với tên data_partition
    output_dir = 'running_outputs'
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(os.path.join(output_dir, 'data_partition.png'))
    plt.close()

    #Lưu ảnh nhiễu vào running_outputs
    # Mean và std từ Normalize
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    # Tạo thư mục lưu ảnh nếu chưa tồn tại
    output_dir = "running_outputs"
    os.makedirs(output_dir, exist_ok=True)

    # Khởi tạo một lưới 10x6 để hiển thị ảnh
    fig, axes = plt.subplots(10, 6, figsize=(15, 25))

    # Duyệt qua 60 trainloaders và hiển thị ảnh đầu tiên
    for i, trainloader in enumerate(trainloaders[:num_partitions]):
        # Lấy ảnh đầu tiên từ trainloader
        image_tensor = trainloader.dataset[0][0].clone()  # Clone để tránh thay đổi dữ liệu gốc
        
        # Tìm vị trí hàng, cột trong lưới
        row, col = divmod(i, 6)
        plt.sca(axes[row, col])  # Đặt trục hiện tại là vị trí hàng, cột trong lưới
        
        # Hiển thị ảnh
        image_numpy = display_image(image_tensor, mean, std)
        axes[row, col].imshow(image_numpy)
        axes[row, col].axis('off')
    plt.title(f"Noise image with sigma from {sigma * 1 / num_partitions} to {sigma}")
    # Điều chỉnh layout để không bị chồng lấn
    plt.tight_layout()

    # Lưu ảnh thay vì hiển thị
    output_path = os.path.join(output_dir, "image_noise.png")
    plt.savefig(output_path, dpi=300)  # Lưu ảnh với chất lượng cao

    plt.close()  # Đóng figure

    print(f"Ảnh đã được lưu tại {output_path}")

    print(f'Number of train samples: {len(trainset)}, val samples: {len(valset)}, test samples: {len(testloader.dataset)}')

###
    return trainloaders, valloaders, testloader


def prepare_quantity_skew_dirichlet(num_partitions: int, batch_size: int, val_ratio: float = 0.1, beta: float = 10, seed: int = 42):
    trainset, testset = get_custom_dataset()
    num_train = int((1 - val_ratio) * len(trainset))
    num_val = len(trainset) - num_train
    trainset, valset = random_split(trainset, [num_train, num_val], generator=torch.Generator().manual_seed(seed))

    all_indices = trainset.indices

    min_size = 0
    while min_size < 1:
        proportions = np.random.dirichlet(np.repeat(beta, num_partitions))
        proportions = (np.cumsum(proportions) * len(all_indices)).astype(int)[:-1]

        partition_indices = np.split(all_indices, proportions)

        min_size = min([len(partition) for partition in partition_indices])
        print('Partition sizes:', [len(partition) for partition in partition_indices])
        print('Min partition size:', min_size)

    trainsets = [Subset(trainset.dataset, indices) for indices in partition_indices]

    partition_len_val = [len(valset) // num_partitions] * num_partitions
    for i in range(len(valset) % num_partitions):
        partition_len_val[i] += 1
    
    valsets = random_split(valset, partition_len_val, generator=torch.Generator().manual_seed(seed))

    trainloaders = [DataLoader(ts, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS) for ts in trainsets]
    valloaders = [DataLoader(vs, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS) for vs in valsets]
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS)

    class_distributions = []
    for i, trainloader in enumerate(trainloaders):
        class_counts = Counter()
        for _, labels in trainloader:
            class_counts.update(labels.numpy())
        class_distributions.append(class_counts)
        print(f'Partition {i} class distribution: {dict(class_counts)}')
    
    partitions = range(num_partitions)
    class_0_counts = [class_distributions[i][0] for i in partitions]
    class_1_counts = [class_distributions[i][1] for i in partitions]

    bar_width = 0.5
    plt.figure(figsize=(12, 8))
    plt.bar(partitions, class_0_counts, bar_width, label='Class 0', color='blue')
    plt.bar(partitions, class_1_counts, bar_width, bottom=class_0_counts, label='Class 1', color='red')
    plt.xlabel('Partition')
    plt.ylabel('Number of Samples')
    plt.title('Class Distribution in Each Partition')
    plt.legend()
    plt.grid(True)
    # plt.show()
    #  Lưu đồ thị vào thư mục running_outputs với tên data_partition
    output_dir = 'running_outputs'
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(os.path.join(output_dir, 'data_partition.png'))
    plt.close()

    print(f'Number of train samples: {len(trainset)}, val samples: {len(valset)}, test samples: {len(testloader.dataset)}')

    return trainloaders, valloaders, testloader


def  load_datasets(
    config: DictConfig,
    num_clients: int,
    val_ratio: float = 0.1,
    seed: Optional[int] = 42,
) -> Tuple[List[DataLoader], List[DataLoader], DataLoader]:
    """Create the dataloaders to be fed into the model.

    Parameters
    ----------
    config: DictConfig
        Parameterises the dataset partitioning process
    num_clients : int
        The number of clients that hold a part of the data
    val_ratio : float, optional
        The ratio of training data that will be used for validation (between 0 and 1),
        by default 0.1
    seed : int, optional
        Used to set a fix seed to replicate experiments, by default 42

    Returns
    -------
    Tuple[DataLoader, DataLoader, DataLoader]
        The DataLoaders for training, validation, and testing.
    """
    print(f"Dataset partitioning config: {config}")
    batch_size = -1
    print('config:' , config)
    if "batch_size" in config:
        batch_size = config.batch_size
    elif "batch_size_ratio" in config:
        batch_size_ratio = config.batch_size_ratio
    else:
        raise ValueError
    partitioning = ""
    
    if "partitioning" in config:
        partitioning = config.partitioning

    # partition the data
    if partitioning == "imbalance_label":
        return prepare_partitioned_dataset(num_clients, batch_size, val_ratio, config.labels_per_client, config.seed)

    if partitioning == "imbalance_label_dirichlet":
        return prepare_imbalance_label_dirichlet(num_clients, batch_size, val_ratio, config.alpha, config.seed)

    if partitioning == "noise_based_imbalance":
        return prepare_noise_based_imbalance(num_clients, batch_size, val_ratio, config.sigma, config.seed)

    if partitioning == "quantity_skew_dirichlet":
        return prepare_quantity_skew_dirichlet(num_clients, batch_size, val_ratio, config.alpha, config.seed)











BACKEND:  Agg


In [8]:
import os
import matplotlib.pyplot as plt

def save_fig_with_incremental_name(base_path):
    if not os.path.exists(base_path):
        plt.savefig(base_path)
    else:
        base_name, ext = os.path.splitext(base_path)
        counter = 1
        new_path = f"{base_name} ({counter}){ext}"
        while os.path.exists(new_path):
            counter += 1
            new_path = f"{base_name} ({counter}){ext}"
        plt.savefig(new_path)
    plt.close()

# Sử dụng hàm
# save_fig_with_incremental_name('running_outputs/accuracy_summary.png')

In [9]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResNet18(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet18, self).__init__()
        self.model = models.resnet18(pretrained=True)
        # self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        self.model.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)  # Output corresponds to num_classes
        )

    def forward(self, x):
        return self.model(x)


class ResNet50(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet50, self).__init__()
        self.model = models.resnet50(pretrained=True)  # Sử dụng ResNet50

        # Điều chỉnh lại lớp phân loại (fc) với cấu trúc mới
        self.model.fc = nn.Sequential(
            nn.Flatten(),                     # Flatten đầu vào
            nn.Linear(2048, 512),             # Chuyển từ 2048 (đặc trưng đầu ra của ResNet50) xuống 512
            nn.ReLU(inplace=True),            # Hàm kích hoạt ReLU
            nn.Linear(512, 128),              # Chuyển từ 512 xuống 128
            nn.ReLU(inplace=True),            # Hàm kích hoạt ReLU
            nn.Linear(128, num_classes)       # Output với số lớp bằng num_classes
        )

    def forward(self, x):
        return self.model(x)  # Truyền dữ liệu qua mô hình ResNet50
    

class VGG11Model(nn.Module):
    # Implement VGG11 model for transfer learning
    def __init__(self, num_classes):
        super().__init__()
        self.model = models.vgg11(pretrained=True)
        
        # Freeze the convolutional base
        # for param in self.model.features.parameters():
        #     param.requires_grad = False
        
        # Replace avgpool with AdaptiveAvgPool2d
        self.model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Replace the classifier with a new one
        self.model.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)  # Output corresponds to num_classes
        )

    def forward(self, x):
        return self.model(x)

In [10]:
import torch
import torch.optim as optim 
import copy
import random 
import numpy as np
import time 
import matplotlib.pyplot as plt

def federated_train(trainloaders, valloaders, testloader, config):
    model = ResNet18(num_classes=2)
    # model = ResNet50(num_classes=2)
    # model = VGG11Model(num_classes=2)
    global_model = copy.deepcopy(model)  # Bản sao mô hình toàn cục
        
    num_rounds = config.num_rounds  # Số vòng huấn luyện
    accs = []

    accs.append(evaluate(global_model, testloader)) #huan luyen 1 lan trc o server
    for round_num in range(num_rounds):
        print(f"Round {round_num + 1}/{num_rounds}")
        start = time.time()
        a_list = []
        d_list = []
        n_list = []
        # Chọn các client tham gia vào mỗi round
        selected_clients = select_clients(trainloaders, config.clients_per_round)
        
        # Huấn luyện trên các client đã chọn
        for client in selected_clients:
            a_i, d_i = local_train(client, global_model, config, trainloaders)
            a_list.append(a_i)
            d_list.append(d_i)
            n_list.append(len(trainloaders[client].dataset))
            
        total_n = sum(n_list)


        d_total_round = copy.deepcopy(global_model.state_dict())
        for key in d_total_round:
            d_total_round[key] = 0.0
        
        for i in range(len(selected_clients)):
            d_para = d_list[i]
            for key in d_para:
                d_total_round[key] += d_para[key] * n_list[i] / total_n
        
        #Update global model
        coeff = 0.0
        for i in range(len(selected_clients)):
            coeff += a_list[i] * n_list[i] / total_n
        
        updated_model = global_model.state_dict()
        for key in updated_model:
            if updated_model[key].type() == 'torch.LongTensor':
                updated_model[key] -= (coeff * d_total_round[key]).type(torch.LongTensor)
            elif updated_model[key].type() == 'torch.cuda.LongTensor':
                    updated_model[key] -= (coeff * d_total_round[key]).type(torch.cuda.LongTensor)
            else:
                #print(updated_model[key].type())
                #print((coeff*d_total_round[key].type()))
                updated_model[key] -= coeff * d_total_round[key]
        global_model.load_state_dict(updated_model)    

        # # Cập nhật mô hình toàn cục sử dụng FedNova
        # global_model = fednova_update(global_model, len_data_local_select, local_deltas, taus)
        
        # Đánh giá mô hình trên tập kiểm tra
        acc = evaluate(global_model, testloader)
        accs.append(acc)
        # Điều chỉnh learning rate theo độ chính xác
        # if acc > 70.0:
        #     config.learning_rate = 1e-5
        #     print(f"Accuracy > 80%, decreasing learning rate to {config.learning_rate}")
        # elif acc > 65.0:
        #     config.learning_rate = 1e-4
        #     print(f"Accuracy > 70%, decreasing learning rate to {config.learning_rate}")
        end = time.time()
        print(f'Time for round {round_num + 1}: ', end-start)
    print('accuracies: ', accs)
    plt.plot(range(0, num_rounds + 1), accs, marker='o', label='Accuracy')
    plt.xlabel('Round')
    plt.xticks(range(0, num_rounds + 1, 10))
    plt.ylabel('Accuracy')
    plt.title('FedNova on ResNet18 over Rounds')
    plt.grid(True)
    plt.legend()
    # plt.savefig('running_outputs/accuracy_summary.png')
    save_fig_with_incremental_name('running_outputs/accuracy_summary.png')
    plt.close()



def select_clients(trainloaders, clients_per_round):
    """Chọn ngẫu nhiên một số client tham gia huấn luyện trong mỗi round."""
    # Số lượng client có sẵn
    total_clients = len(trainloaders)
    # Chọn ngẫu nhiên một số client
    selected_clients = random.sample(range(total_clients), clients_per_round)
    return selected_clients

def local_train(client, global_model, config, trainloader):
    """Huấn luyện mô hình trên một client cụ thể."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = copy.deepcopy(global_model).to(device) # Sử dụng bản sao mô hình toàn cục
    net.train()
    
    # Sử dụng SGD với learning rate = 1e-3 và momentum = 0.9
    optimizer = optim.SGD(
        # net.parameters(),
        filter(lambda p: p.requires_grad, net.parameters()), 
        lr=config.learning_rate, 
        momentum=config.momentum)
    tau = 0
    print(f"Training on client {client}, device: {device}, learning_rate={config.learning_rate}")
    # Huấn luyện mô hình trên client
    for epoch in range(config.num_epochs):
        for batch_idx, (data, target) in enumerate(trainloader[client]):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = net(data)
            loss = torch.nn.CrossEntropyLoss()(output, target)
            loss.backward()
            optimizer.step()
            tau += 1  # Tăng số lần cập nhật

    a_i = (tau-config.momentum * (1-pow(config.momentum, tau)) / (1 - config.momentum)) / (1 - config.momentum)
    global_model_para = global_model.state_dict()
    net_para = net.state_dict()
    norm_grad = copy.deepcopy(global_model.state_dict())
    for key in norm_grad:
        norm_grad[key] = torch.true_divide(global_model_para[key] - net_para[key].to('cpu'), a_i)
    

    return a_i, norm_grad

def evaluate(model, testloader):
    """Đánh giá mô hình trên tập kiểm tra."""
    model.eval()  # Chuyển sang chế độ đánh giá
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in testloader:
            output = model(data)
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy


In [11]:
from omegaconf import OmegaConf  # Thêm OmegaConf
import os

# def load_config(config_file):
#     """Load configuration using OmegaConf (DictConfig)"""
#     # Dùng OmegaConf để load file config.yaml
#     config = OmegaConf.load(config_file)
#     return config
def load_config():
    """Load configuration using OmegaConf from a dictionary."""
    config_dict = {
        "num_clients": 4,
        "num_epochs": 1,
        "batch_size": 10,
        "clients_per_round": 2,
        "fraction_fit": 0.1,
        "learning_rate": 1e-3,
        "num_rounds": 1,
        "partitioning": "imbalance_label",
        "dataset_name": "chest_xray",
        "dataset_seed": 42,
        "alpha": 0.5,
        "sigma": 0.1,
        "labels_per_client": 1,  # only used when partitioning is label quantity
        "momentum": 0.9,
        "weight_decay": 0.00001,
        "dataset": {
            "name": "${dataset_name}",
            "partitioning": "${partitioning}",
            "batch_size": "${batch_size}",  # batch_size = batch_size_ratio * total_local_data_size
            "val_split": 0.0,
            "seed": "${dataset_seed}",
            "alpha": "${alpha}",
            "sigma": "${sigma}",
            "labels_per_client": "${labels_per_client}"
        }
    }

    # Chuyển đổi dictionary thành DictConfig
    config = OmegaConf.create(config_dict)

    return config

def main():
    # Parse arguments
    # args = parse_args()

    # Load configuration file
    config = load_config()  # Trả về DictConfig

    # Kiểm tra các tham số được thay thế chính xác
    print("Loaded Config:")
    print(config)

    # Load dataset
    # trainloaders, valloaders, testloader = load_datasets(config.dataset.name, args.num_clients)
    trainloaders, valloaders, testloader = load_datasets(
        config=config.dataset,
        num_clients=config.num_clients,
        val_ratio=config.dataset.val_split,
    )

    # Train federated model
    federated_train(trainloaders, valloaders, testloader, config)

if __name__ == '__main__':
    main()

Loaded Config:
{'num_clients': 4, 'num_epochs': 1, 'batch_size': 10, 'clients_per_round': 2, 'fraction_fit': 0.1, 'learning_rate': 0.001, 'num_rounds': 1, 'partitioning': 'imbalance_label', 'dataset_name': 'chest_xray', 'dataset_seed': 42, 'alpha': 0.5, 'sigma': 0.1, 'labels_per_client': 1, 'momentum': 0.9, 'weight_decay': 1e-05, 'dataset': {'name': '${dataset_name}', 'partitioning': '${partitioning}', 'batch_size': '${batch_size}', 'val_split': 0.0, 'seed': '${dataset_seed}', 'alpha': '${alpha}', 'sigma': '${sigma}', 'labels_per_client': '${labels_per_client}'}}
Dataset partitioning config: {'name': '${dataset_name}', 'partitioning': '${partitioning}', 'batch_size': '${batch_size}', 'val_split': 0.0, 'seed': '${dataset_seed}', 'alpha': '${alpha}', 'sigma': '${sigma}', 'labels_per_client': '${labels_per_client}'}
config: {'name': '${dataset_name}', 'partitioning': '${partitioning}', 'batch_size': '${batch_size}', 'val_split': 0.0, 'seed': '${dataset_seed}', 'alpha': '${alpha}', 'sigma'

/media/namvq/Data/anaconda3/envs/env1/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/media/namvq/Data/anaconda3/envs/env1/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test Accuracy: 46.51%
Round 1/1
Training on client 0, device: cuda, learning_rate=0.001
Training on client 1, device: cuda, learning_rate=0.001
Test Accuracy: 46.51%
Time for round 1:  9.408247947692871
accuracies:  [46.51162790697674, 46.51162790697674]


In [12]:
from omegaconf import OmegaConf  # Thêm OmegaConf
import os

# def load_config(config_file):
#     """Load configuration using OmegaConf (DictConfig)"""
#     # Dùng OmegaConf để load file config.yaml
#     config = OmegaConf.load(config_file)
#     return config
def load_config():
    """Load configuration using OmegaConf from a dictionary."""
    config_dict = {
        "num_clients": 4,
        "num_epochs": 1,
        "batch_size": 10,
        "clients_per_round": 2,
        "fraction_fit": 0.1,
        "learning_rate": 1e-3,
        "num_rounds": 2,
        "partitioning": "imbalance_label",
        "dataset_name": "chest_xray",
        "dataset_seed": 42,
        "alpha": 0.5,
        "sigma": 0.1,
        "labels_per_client": 1,  # only used when partitioning is label quantity
        "momentum": 0.9,
        "weight_decay": 0.00001,
        "dataset": {
            "name": "${dataset_name}",
            "partitioning": "${partitioning}",
            "batch_size": "${batch_size}",  # batch_size = batch_size_ratio * total_local_data_size
            "val_split": 0.0,
            "seed": "${dataset_seed}",
            "alpha": "${alpha}",
            "sigma": "${sigma}",
            "labels_per_client": "${labels_per_client}"
        }
    }

    # Chuyển đổi dictionary thành DictConfig
    config = OmegaConf.create(config_dict)

    return config

def main():
    # Parse arguments
    # args = parse_args()

    # Load configuration file
    config = load_config()  # Trả về DictConfig

    # Kiểm tra các tham số được thay thế chính xác
    print("Loaded Config:")
    print(config)

    # Load dataset
    # trainloaders, valloaders, testloader = load_datasets(config.dataset.name, args.num_clients)
    trainloaders, valloaders, testloader = load_datasets(
        config=config.dataset,
        num_clients=config.num_clients,
        val_ratio=config.dataset.val_split,
    )

    # Train federated model
    federated_train(trainloaders, valloaders, testloader, config)

if __name__ == '__main__':
    main()

Loaded Config:
{'num_clients': 4, 'num_epochs': 1, 'batch_size': 10, 'clients_per_round': 2, 'fraction_fit': 0.1, 'learning_rate': 0.001, 'num_rounds': 2, 'partitioning': 'imbalance_label', 'dataset_name': 'chest_xray', 'dataset_seed': 42, 'alpha': 0.5, 'sigma': 0.1, 'labels_per_client': 1, 'momentum': 0.9, 'weight_decay': 1e-05, 'dataset': {'name': '${dataset_name}', 'partitioning': '${partitioning}', 'batch_size': '${batch_size}', 'val_split': 0.0, 'seed': '${dataset_seed}', 'alpha': '${alpha}', 'sigma': '${sigma}', 'labels_per_client': '${labels_per_client}'}}
Dataset partitioning config: {'name': '${dataset_name}', 'partitioning': '${partitioning}', 'batch_size': '${batch_size}', 'val_split': 0.0, 'seed': '${dataset_seed}', 'alpha': '${alpha}', 'sigma': '${sigma}', 'labels_per_client': '${labels_per_client}'}
config: {'name': '${dataset_name}', 'partitioning': '${partitioning}', 'batch_size': '${batch_size}', 'val_split': 0.0, 'seed': '${dataset_seed}', 'alpha': '${alpha}', 'sigma'